In [ ]:
import math
from collections import namedtuple
import os
import re
import datetime as dt
import json

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from pprint import pprint

%matplotlib inline

Define some helper methods and data structures

In [ ]:
GeoExtent = namedtuple('GeoExtent', ['x_min', 'y_max', 'x_max', 'y_min'])
GeoAffine = namedtuple('GeoAffine', ['ul_x', 'x_res', 'rot_1', 'ul_y', 'rot_2', 'y_res'])
GeoCoordinate = namedtuple('GeoCoordinate', ['x', 'y'])
RowColumn = namedtuple('RowColumn', ['row', 'column'])
RowColumnExtent = namedtuple('RowColumnExtent', ['start_row', 'start_col', 'end_row', 'end_col'])

In [ ]:
def geospatial_hv(h, v, loc):
    """
    Geospatial extent and 30m affine for a given ARD grid location.
    """
    xmin = loc.x_min + h * 5000 * 30
    xmax = loc.x_min + h * 5000 * 30 + 5000 * 30
    ymax = loc.y_max - v * 5000 * 30
    ymin = loc.y_max - v * 5000 * 30 - 5000 * 30

    return (GeoExtent(x_min=xmin, x_max=xmax, y_max=ymax, y_min=ymin),
            GeoAffine(ul_x=xmin, x_res=30, rot_1=0, ul_y=ymax, rot_2=0, y_res=-30))

In [ ]:
def geo_to_rowcol(affine, coord):
    """
    Transform geo-coordinate to row/col given a reference affine.
    
    Yline = (Ygeo - GT(3) - Xpixel*GT(4)) / GT(5)
    Xpixel = (Xgeo - GT(0) - Yline*GT(2)) / GT(1)
    """
    row = (coord.y - affine.ul_y - affine.ul_x * affine.rot_2) / affine.y_res
    col = (coord.x - affine.ul_x - affine.ul_y * affine.rot_1) / affine.x_res

    return RowColumn(row=int(row),
                     column=int(col))

In [ ]:
def rowcol_to_geo(affine, rowcol):
    """
    Transform a row/col into a geospatial coordinate given reference affine.
    
    Xgeo = GT(0) + Xpixel*GT(1) + Yline*GT(2)
    Ygeo = GT(3) + Xpixel*GT(4) + Yline*GT(5)
    """
    x = affine.ul_x + rowcol.column * affine.x_res + rowcol.row * affine.rot_1
    y = affine.ul_y + rowcol.column * affine.rot_2 + rowcol.row * affine.y_res

    return GeoCoordinate(x=x, y=y)

In [ ]:
def load_cache(file):
    """
    Load the cache file and split the data into the image IDs and values
    """
    data = np.load(file)
    return data['Y'], data['image_IDs']

In [ ]:
def find_file(file_ls, string):
    """
    Return the first str in a list of strings that contains string.
    """
    gen = filter(lambda x: string in x, file_ls)
    return next(gen, None)

In [ ]:
def imageid_date(image_ids):
    """
    Extract the ordinal day from the ARD image name.
    """
    return np.array([dt.datetime.strptime(d[15:23], '%Y%m%d').toordinal()
                     for d in image_ids])

In [ ]:
def mask_daterange(dates):
    """
    Create a mask for values outside of the global BEGIN_DATE and END_DATE.
    """
    mask = np.zeros_like(dates, dtype=bool)
    
    mask = np.logical_and(dates >= BEGIN_DATE.toordinal(), dates <= END_DATE.toordinal())
    
    # mask[(dates >= BEGIN_DATE.toordinal()) & (dates <= END_DATE.toordinal())] = 1

    return mask

In [ ]:
def mask_daterange_edit(dates):
    """
    Create a mask for values outside of the global BEGIN_DATE and END_DATE.
    Create a similar mask for values inside of these global values.
    """
    mask_in = np.zeros_like(dates, dtype=bool)
    mask_out = np.copy(mask_in)
    
    mask_in = np.logical_and(dates >= BEGIN_DATE.toordinal(), dates <= END_DATE.toordinal())
    
    # mask_in[dates >= BEGIN_DATE.toordinal() & (dates <= END_DATE.toordinal())] = 1
    
    mask_out = np.logical_or(dates < BEGIN_DATE.toordinal(), dates > END_DATE.toordinal())
    
    # mask_out[(dates < BEGIN_DATE.toordinal()) | (dates > END_DATE.toordinal())] = 1

    return mask_in, mask_out

In [ ]:
def find_chipcurve(results_chip, coord):
    """
    Find the results for the specified coordinate.
    """
    with open(results_chip, 'r') as f:
        results = json.load(f)
    
    gen = filter(lambda x: coord.x == x['x'] and coord.y == x['y'], results)
    
    return next(gen, None)

In [ ]:
def extract_cachepoint(coord):
    """
    Extract the spectral values from the cache file.
    """

    rowcol = geo_to_rowcol(PIXEL_AFFINE, coord)
    
    data, image_ids = load_cache(find_file(CACHE_INV, 'r{}'.format(rowcol.row)))
    
    dates = imageid_date(image_ids)
        
    # mask = mask_daterange(dates)
    
    return image_ids, data[:, :, rowcol.column], dates
   

In [ ]:
def extract_cachepoint_edit(coord):
    """
    Extract the spectral values from the cache file.
    """

    rowcol = geo_to_rowcol(PIXEL_AFFINE, coord)
    
    data, image_ids = load_cache(find_file(CACHE_INV, 'r{}'.format(rowcol.row)))
    
    dates = imageid_date(image_ids)
    
    # Duplicate dates removed
    dates_, indices = np.unique(dates, return_index=True)
    
    data_ = data[:, indices]
    
    mask_in, mask_out = mask_daterange_edit(dates_)
    
    
    return image_ids, data[:, mask, rowcol.column], dates[mask], mask_in, mask_out
    
    """
    # Check if the len of the processing mask equals the len of dates with duplicates removed
    # For most cases the length of the internal processing mask should be equal to the 
    # number of observations within the BEGIN and END date range
    
    if len(results) == len(dates_[mask_in]):

        print("The length of the pyccd internal processing mask ({}) is consistent with the number of observations"
              " in the cache files ({}) with duplicate dates removed".format(len(results),
                                                                             len(dates_[mask_in])))

        # mask_in, mask_out = mask_daterange(dates_)

        return image_ids, data_[:, mask_in, rowcol.column], dates_[mask_in], \
               data_[:, mask_out, rowcol.column], dates_[mask_out]

    elif len(results) != len(dates_[mask_in]):

        mask_in, mask_out = self.mask_daterange_edit(dates)

        if len(results) == len(dates[mask_in]):

            print("The length of the pyccd internal processing mask ({}) is consistent with the "
                  "number of observations in the cache files ({}) if duplicate dates are not "
                  "removed".format(len(results), len(dates[mask_in])))

            return image_ids, data[:, mask_in, rowcol.column], dates[mask_in], \
                   data[:, mask_out, rowcol.column], dates[mask_out]

        else:

            print("The length of the pyccd internal processing mask ({}) is inconsistent with"
                  " the number of observations provided in the cache files ({})".format(len(results),
                                                                                        len(dates[mask_in])))

            sys.exit(1)

    return None
    """
   

In [ ]:
def extract_jsoncurve(coord):
    """
    Extract the pyccd information from the json file representing a chip of results.
    """
    pixel_rowcol = geo_to_rowcol(PIXEL_AFFINE, coord)
    pixel_coord = rowcol_to_geo(PIXEL_AFFINE, pixel_rowcol)
    
    chip_rowcol = geo_to_rowcol(CHIP_AFFINE, coord)
    chip_coord = rowcol_to_geo(CHIP_AFFINE, chip_rowcol)
    
    file = find_file(JSON_INV, 'H{:02d}V{:02d}_{}_{}.json'.format(H, V, chip_coord.x, chip_coord.y))
    result = find_chipcurve(file, pixel_coord)
    
    if result.get('result_ok') is True:
        return json.loads(result['result'])

In [ ]:
def predicts(days, coef, intercept):
    return (intercept + coef[0] * days +
            coef[1]*np.cos(days*1*2*np.pi/365.25) + coef[2]*np.sin(days*1*2*np.pi/365.25) +
            coef[3]*np.cos(days*2*2*np.pi/365.25) + coef[4]*np.sin(days*2*2*np.pi/365.25) +
            coef[5]*np.cos(days*3*2*np.pi/365.25) + coef[6]*np.sin(days*3*2*np.pi/365.25))

In [ ]:
def arcpaste_to_coord(string):
    pieces = string.split()
    
    return GeoCoordinate(x=float(re.sub(',', '', pieces[0])),
                         y=float(re.sub(',', '', pieces[1])))

Setup file locations

In [ ]:
JSON_DIR = r'Z:\sites\sd\pyccd-results\H13V06\2017.08.18\json'
JSON_INV = [os.path.join(JSON_DIR, f) for f in os.listdir(JSON_DIR)]
CACHE_DIR = r'Z:\sites\sd\ARD\h13v06\cache'
CACHE_INV = [os.path.join(CACHE_DIR, f) for f in os.listdir(CACHE_DIR)]

In [ ]:
arc_paste = '-549,940.134  2,350,557.910 Meters'
coord = arcpaste_to_coord(arc_paste)

CONUS_EXTENT = GeoExtent(x_min=-2565585,
                         y_min=14805,
                         x_max=2384415,
                         y_max=3314805)

H = 13
V = 6
EXTENT, PIXEL_AFFINE = geospatial_hv(H, V, CONUS_EXTENT)
CHIP_AFFINE = GeoAffine(ul_x=PIXEL_AFFINE.ul_x, x_res=3000, rot_1=0, ul_y=PIXEL_AFFINE.ul_y, rot_2=0, y_res=-3000)



In [ ]:
results = extract_jsoncurve(coord)


In [ ]:
BEGIN_DATE = dt.datetime.fromordinal(results["change_models"][0]["start_day"])
END_DATE = dt.datetime.fromordinal(results["change_models"][-1]["end_day"])

print(BEGIN_DATE, END_DATE)

In [ ]:
imageIDs, data, dates = extract_cachepoint(coord)

In [ ]:
# rescale the brightness temperature to match the predicted values
temp_thermal_data = np.copy(data[6])
temp_thermal_data[ temp_thermal_data != -9999 ] = temp_thermal_data[ temp_thermal_data != -9999 ] * 10 - 27315
data[6] = np.copy(temp_thermal_data)

Setup geospatial and temporal information

In [ ]:
print(np.shape(dates))
print(len(results["processing_mask"]))

date_mask = mask_daterange(dates=dates)
dates_masked = dates[date_mask]

In [ ]:
print(len(dates_masked))

In [ ]:
print(BEGIN_DATE, END_DATE)

In [ ]:
print(results.keys())

In [ ]:
print(len(results["processing_mask"]), len(results["change_models"]))

In [ ]:
print(len(dates), len(data[0]))

In [ ]:
qa = data[-1]

In [ ]:
# Make a mask based on the ARD QA band to remove fill (value 1)
qa_mask = np.ones_like(qa, dtype=np.bool)
qa_mask[qa == 1] = False
qa_mask

In [ ]:
results[]

In [ ]:
def msavi(R, NIR):
    # Modified Soil Adjusted Vegetation Index
    
    return (2.0 * NIR + 1.0 - ((2.0 * NIR + 1.0)**2.0 - 8.0 * (NIR - R))**0.5) / 2.0

def ndvi(R, NIR):
    # Normalized Difference Vegetation Index
    
    return (NIR - R) / (NIR + R)

def evi(B, R, NIR, G=2.5, L=1.0, C1=6, C2=7.5):
    # Enhanced Vegetation Index
    
    return G * ((NIR - R) / (NIR + C1 * R - C2 * B + L))

def savi(R, NIR, L=0.5):
    # Soil Adjusted Vegetation Index
    
    return ((NIR - R) / (NIR + R + L)) * (1 + L)

def ndmi(NIR, SWIR1):
    # Normalized Difference Moisture Index
    
    return (NIR - SWIR1) / (NIR + SWIR1)

def nbr(NIR, SWIR2):
    # Normalized Burn Ratio
    
    return (NIR - SWIR2) / (NIR + SWIR2)

def nbr2(SWIR1, SWIR2):
    # Normalized Burn Ratio 2
    
    return (SWIR1 - SWIR2) / (SWIR1 + SWIR2)  
    

In [ ]:
bands = ('blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'thermal')
band_info = {b: {'coefs': [], 'inter': [], 'pred': []} for b in bands}

#mask = np.array(results['processing_mask'], dtype=bool)
mask = np.ones_like(dates, dtype=bool)
mask[: len(results["processing_mask"])] = results["processing_mask"]

"""
print('Start Date: {0}\nEnd Date: {1}\n'.format(dt.datetime.fromordinal(dates[0]),
                                                dt.datetime.fromordinal(dates[-1])))
"""

predicted_values = []
prediction_dates = []
break_dates = []
start_dates = []

# get year values for labeling plots
year1 = str(dt.datetime.fromordinal(dates[0]))[:4]
year2 = str(dt.datetime.fromordinal(dates[-1]))[:4]
years = np.arange(int(year1), int(year2), 2)

for num, result in enumerate(results['change_models']):
    print('Result: {}'.format(num))
    print('Start Date: {}'.format(dt.date.fromordinal(result['start_day'])))
    print('End Date: {}'.format(dt.date.fromordinal(result['end_day'])))
    print('Break Date: {}'.format(dt.date.fromordinal(result['break_day'])))
    print('QA: {}'.format(result['curve_qa']))
    print('Change prob: {}'.format(result['change_probability']))
    
    days = np.arange(result['start_day'], result['end_day'] + 1)
    # prediction_dates.append(days)
    break_dates.append(result['break_day'])
    start_dates.append(result['start_day'])
    
    for b in bands:
        band_info[b]['inter'] = result[b]['intercept']
        band_info[b]['coefs'] = result[b]['coefficients']
        band_info[b]['pred'] = predicts(days, result[b]['coefficients'], result[b]['intercept'])
    
        intercept = result[b]['intercept']
        coef = result[b]['coefficients']
        prediction_dates.append(days)
        predicted_values.append(predicts(days, coef, intercept))
    

plt.style.use('ggplot')


qa = np.copy(data[7,:])
maskqa = np.ones_like(qa, dtype=bool)
maskqa[ qa == 1 ] = False

maskqa = maskqa[mask]

dates_plt = dates[mask]

# ****X-Axis Ticks and Labels****
# list of years
y = [yi for yi in range(1981, 2018, 2)]

# list of datetime objects with YYYY-MM-dd pattern
t = [dt.datetime(yx, 7, 1) for yx in y]

# list of ordinal time objects
ord_time = [dt.datetime.toordinal(tx) for tx in t]

# list of datetime formatted strings
x_labels = [str(dt.datetime.fromordinal(int(L)))[:10] if L != "0.0" and L != "" else "0" for L in ord_time]


for num, b in enumerate(bands):
    fg = plt.figure(figsize=(16,9), dpi=300)
    a1 = fg.add_subplot(2, 1, 1, xlim=(min(dates)-100, max(dates)+500), ylim=(-1000,5000))
    
    
    data_plt = data[num, mask]
    
    # Observed values
    a1.plot(dates_plt[maskqa], data_plt[maskqa], 'go', ms=7, mec='k', mew=0.5) 
    
    # Observed values masked out
    a1.plot(dates[~mask], data[num, ~mask], color="0.65", marker="o", linewidth=0, ms=3)
    
    a1.set_title('Band {}'.format(str(num+1)))
    
    # plot model break and start dates
    for b in break_dates: a1.axvline(b, color='r')
    for s in start_dates: a1.axvline(s, color='b')
    
    # Predicted curves
    for c in range(0 , len(results["change_models"])):
        a1.plot(prediction_dates[c * len(bands) + num], predicted_values[c * len(bands) + num],
               "orange", linewidth=2)

    # Add x-ticks and x-tick_labels 
    a1.set_xticks(ord_time)

    a1.set_xticklabels(x_labels, rotation=70, horizontalalignment="right")



In [ ]:
# plot all observations
NDVI_all = (data[3] - data[2]) / (data[3] + data[2])
fg = plt.figure(figsize=(16,9), dpi=300)
a1 = fg.add_subplot(2, 1, 1)
a1.set_xticks(ord_time)

a1.set_xticklabels(x_labels, rotation=70, horizontalalignment="right")

# don't plot values where NDVI=0
a1.plot(dates[NDVI_all!=0], NDVI_all[NDVI_all!=0], marker="o", linewidth=0, mec="k", mew=0.3)

In [ ]:
# Plot NDVI clear observations
BAND3 = data[2, mask]
BAND4 = data[3, mask]
NDVI = (BAND4 - BAND3) / (BAND3 + BAND4)

fg = plt.figure(figsize=(16,9), dpi=300)
a1 = fg.add_subplot(2,1,1)

# ****X-Axis Ticks and Labels****
# list of years
# y = [yi for yi in range(1981, 2018, 2)]

# get year values for labeling plots
year1 = str(dt.datetime.fromordinal(dates[0]))[:4]
year2 = str(dt.datetime.fromordinal(dates[-1]))[:4]
years = np.arange(int(year1), int(year2) + 2, 2)

# list of datetime objects with YYYY-MM-dd pattern
t = [dt.datetime(yx, 7, 1) for yx in years]

# list of ordinal time objects
ord_time = [dt.datetime.toordinal(tx) for tx in t]

# list of datetime formatted strings
x_labels = [str(dt.datetime.fromordinal(int(L)))[:10] if L != "0.0" and L != "" else "0" for L in ord_time]

a1.set_xticks(ord_time)
a1.set_xticklabels(x_labels, rotation=70, horizontalalignment="right")

a1.plot(dates[mask], NDVI,  marker="o", linewidth=0, mec="k", mew=0.3)

In [ ]:
# Plot NDVI clear observations
BAND3 = data[2, mask]
BAND4 = data[3, mask]
NDVI = (BAND4 - BAND3) / (BAND3 + BAND4)

fg = plt.figure(figsize=(16,9), dpi=300)
a1 = fg.add_subplot(2,1,1)

# ****X-Axis Ticks and Labels****
# list of years generated with static values, I think I prefer the method above that allows for different scenarios of end years
y = [yi for yi in range(1981, 2018, 2)]

# list of datetime objects with YYYY-MM-dd pattern
t = [dt.datetime(yx, 7, 1) for yx in y]

# list of ordinal time objects
ord_time = [dt.datetime.toordinal(tx) for tx in t]

# list of datetime formatted strings
x_labels = [str(dt.datetime.fromordinal(int(L)))[:10] if L != "0.0" and L != "" else "0" for L in ord_time]

a1.set_xticks(ord_time)
a1.set_xticklabels(x_labels, rotation=70, horizontalalignment="right")

a1.plot(dates[mask], NDVI,  marker="o", linewidth=0, mec="k", mew=0.3)

In [ ]:
# calculate the predicted NDVI from PyCCD change models

# for c in range(0 , len(results["change_models"])):
    
    #NDVI_pred.append((predicted_values[c * len(bands) + 3] - predicted_values[c * len(bands) + 2]) / \
    #(predicted_values[c * len(bands) + 3] + predicted_values[c * len(bands) + 2]))
    

# using list comprehensions
numerator = [(predicted_values[c * len(bands) + 3] - predicted_values[c * len(bands) + 2]) for c in range(0 , len(results["change_models"]))]

denominator = [(predicted_values[c * len(bands) + 3] + predicted_values[c * len(bands) + 2]) for c in range(0 , len(results["change_models"]))]

NDVI_pred = [numerator[c] / denominator[c]  for c in range(0 , len(results["change_models"])) if not np.any(denominator[c] == 0)]
    
# Enforce NDVI value range -1 to 1
#for ndvi in NDVI_pred:
#    for index, val in enumerate(ndvi):
#        if val > 1.0: ndvi[index] = 1.0
            
#        elif val < -1.0: ndvi[index] = -1.0


In [ ]:
fg = plt.figure(figsize=(16,9), dpi=300 )
a1 = fg.add_subplot(2,1,1, ylim=(-1.10, 1.10))

# ****X-Axis Ticks and Labels****
# y = [yi for yi in range(1981, 2018, 2)]

# get year values for labeling plots
year1 = str(dt.datetime.fromordinal(dates[0]))[:4]
year2 = str(dt.datetime.fromordinal(dates[-1]))[:4]
years = np.arange(int(year1), int(year2) + 2, 2)

# list of datetime objects with YYYY-MM-dd pattern
t = [dt.datetime(yx, 7, 1) for yx in years]

# list of ordinal time objects
ord_time = [dt.datetime.toordinal(tx) for tx in t]

# list of datetime formatted strings
x_labels = [str(dt.datetime.fromordinal(int(L)))[:10] if L != "0.0" and L != "" else "0" for L in ord_time]

a1.set_title("NDVI")

a1.set_xticks(ord_time)
a1.set_xticklabels(x_labels, rotation=70, horizontalalignment="right")


a1.plot(dates[NDVI_all!=0], NDVI_all[NDVI_all!=0], color="0.65", marker="o", linewidth=0, ms = 3)
a1.plot(dates[mask], NDVI,  marker="o", color="green",linewidth=0, mec="k", mew=0.3)

# plot model break and start dates
for b in break_dates: a1.axvline(b, color='r')
for s in start_dates: a1.axvline(s, color='b')
    
for c in range(0, len(results["change_models"])):
    t1 = NDVI_pred[c] >= -1.0
    t2 = NDVI_pred[c] <= 1.0
    tt = (t1==True) == t2
    a1.plot(prediction_dates[c*len(bands)][tt], NDVI_pred[c][tt],"orange", linewidth=2)
    


In [ ]:
EVI_pred = [2.5 * ((predicted_values[c * len(bands) + 3] - predicted_values[c * len(bands) + 2]) / 
                  (predicted_values[c * len(bands) + 3] + 6 * predicted_values[c * len(bands) + 2] - 7.5 *
                  predicted_values[c * len(bands)] + 1)) for c in range(0, len(results["change_models"]))]

# Plot EVI for clear observations with PyCCD curve
EVI_all = 2.5 * ((data[3] - data[2]) / (data[3] + 6 * data[2] - 7.5 * data[0] + 1))

# Get the band 1 clear observations (already have band 3 and 4 from above)
BAND1 = data[0, mask]

EVI = 2.5 * ((BAND4 - BAND3) / (BAND4 + 6 * BAND3 - 7.5 * BAND1 + 1))

for ind, e in enumerate(EVI_all):
    if e > 1.0: EVI_all[ind] = 0
    elif e < -1.0: EVI_all[ind] = 0
        
for ind, e in enumerate(EVI):
    if e > 1.0: EVI[ind] = 0
    elif e < -1.0: EVI[ind] = 0
        
# Enforce EVI value range -1 to 1
# for evi in EVI_pred:
#    for index, val in enumerate(evi):
#        if val > 1.0: evi[index] = 1.0
            
#        elif val < -1.0: evi[index] = -1.0

In [ ]:
fg = plt.figure(figsize=(16,9), dpi=300)
a1 = fg.add_subplot(2,1,1, ylim=(-1.1, 1.1))

# ****X-Axis Ticks and Labels****
# y = [yi for yi in range(1981, 2018, 2)]

# get year values for labeling plots
year1 = str(dt.datetime.fromordinal(dates[0]))[:4]
year2 = str(dt.datetime.fromordinal(dates[-1]))[:4]
years = np.arange(int(year1), int(year2) + 2, 2)

# list of datetime objects with YYYY-MM-dd pattern
t = [dt.datetime(yx, 7, 1) for yx in years]

# list of ordinal time objects
ord_time = [dt.datetime.toordinal(tx) for tx in t]

# list of datetime formatted strings
x_labels = [str(dt.datetime.fromordinal(int(L)))[:10] if L != "0.0" and L != "" else "0" for L in ord_time]

a1.set_title("EVI")

a1.set_xticks(ord_time)
a1.set_xticklabels(x_labels, rotation=70, horizontalalignment="right")

a1.plot(dates[~mask][EVI_all[~mask] != 0], EVI_all[~mask][EVI_all[~mask] != 0], color="0.65", marker="o", linewidth=0, ms=3)
a1.plot(dates[mask][EVI !=0], EVI[EVI!=0],  marker="o", color="green",linewidth=0, mec="k", mew=0.3)

# plot model break and start dates
for b in break_dates: a1.axvline(b, color='r')
for s in start_dates: a1.axvline(s, color='b')

for c in range(0, len(results["change_models"])):
    t1 = EVI_pred[c] >= -1.0
    t2 = EVI_pred[c] <= 1.0
    tt = (t1==True) == t2
    a1.plot(prediction_dates[c*len(bands)][tt], EVI_pred[c][tt] ,"orange", linewidth=2)
    #a1.plot(prediction_dates[c*len(bands)], EVI_pred[c] , linewidth=2)
    


In [ ]:
SAVI_pred = [savi(R=predicted_values[c * len(bands) + 3], NIR=predicted_values[c * len(bands) + 2]) 
            for c in range(0, len(results["change_models"]))]

SAVI_all = savi(R=data[2], NIR=data[3])

SAVI_clear = savi(R=BAND3, NIR=BAND4)


In [ ]:
fg = plt.figure(figsize=(16,9), dpi=300)
a1 = fg.add_subplot(2,1,1, ylim=(-1.1, 1.1))

# ****X-Axis Ticks and Labels****
# y = [yi for yi in range(1981, 2018, 2)]

# get year values for labeling plots
year1 = str(dt.datetime.fromordinal(dates[0]))[:4]
year2 = str(dt.datetime.fromordinal(dates[-1]))[:4]
years = np.arange(int(year1), int(year2) + 2, 2)

# list of datetime objects with YYYY-MM-dd pattern
t = [dt.datetime(yx, 7, 1) for yx in years]

# list of ordinal time objects
ord_time = [dt.datetime.toordinal(tx) for tx in t]

# list of datetime formatted strings
x_labels = [str(dt.datetime.fromordinal(int(L)))[:10] if L != "0.0" and L != "" else "0" for L in ord_time]

a1.set_title("SAVI")

a1.set_xticks(ord_time)
a1.set_xticklabels(x_labels, rotation=70, horizontalalignment="right")

a1.plot(dates[~mask][SAVI_all[~mask] != 0], SAVI_all[~mask][SAVI_all[~mask] != 0], color="0.65", marker="o", linewidth=0, ms=3)
a1.plot(dates[mask], SAVI,  marker="o", color="green",linewidth=0, mec="k", mew=0.3)

# plot model break and start dates
for b in break_dates: a1.axvline(b, color='r')
for s in start_dates: a1.axvline(s, color='b')

for c in range(0, len(results["change_models"])):
    t1 = SAVI_pred[c] >= -1.0
    t2 = SAVI_pred[c] <= 1.0
    tt = (t1==True) == t2
    a1.plot(prediction_dates[c*len(bands)][tt], SAVI_pred[c][tt] ,"orange", linewidth=2)
    #a1.plot(prediction_dates[c*len(bands)], SAVI_pred[c] , linewidth=2)
    

In [ ]:
# Plot MSAVI for clear observations with predicted MSAVI

MSAVI_pred = [msavi(R=predicted_values[c * len(bands) + 2], NIR=predicted_values[c * len(bands) + 3]) for c in range(0, len(results["change_models"]))]

In [ ]:
MSAVI_clear = msavi(R=BAND4, NIR=BAND3)
print(len(MSAVI_clear))
MSAVI_all = msavi(data[3, ~mask], data[2, ~mask])
print(len(MSAVI_all))
print(len(mask))

In [ ]:
# datesYMD = [str(dt.datetime.fromordinal(d))[:10] for d in dates]
# datesYMD=[d.replace("-","") for d in datesYMD]

In [ ]:
#for i, j in zip(imageIDs, datesYMD):
   # if i[15:23] != (j): print(i , j)

In [ ]:
# len(imageIDs), len(datesYMD)

In [ ]:
#np.shape(data)

In [ ]:
#qa = np.copy(data[7,:])

In [ ]:
#np.shape(qa)

In [ ]:
#np.amin(qa)

In [ ]:
#np.amax(qa)

In [ ]:
#len(data)

In [ ]:
#qa_vals = [(a, ":", bin(a)) for a in np.unique(qa)]
#for q in qa_vals: print(q,'\n')